In [ ]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../fuzzy-torch') # or just install the module
sys.path.append('../../fuzzy-tools') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [ ]:
%load_ext autoreload
%autoreload 2
from lcclassifier.results.utils import get_model_names

rootdir = '../save'
lcset_name = 'test'
method = 'spm-mcmc-estw'
cfilename = f'survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method={method}'
kf = '2'
mid = '1000'

model_names = get_model_names(rootdir, cfilename, kf, lcset_name)
model_names

In [ ]:
%load_ext autoreload
%autoreload 2
import fuzzytools.files as files
import matplotlib.pyplot as plt
from fuzzytools.matplotlib.scatters import scatter
from fuzzytools.matplotlib.lims import AxisLims
from fuzzytools.matplotlib.animators import PlotAnimator
import numpy as np
from lcclassifier._C import CLASSES_STYLES
from copy import copy, deepcopy
from lchandler.results import utils as utils

for model_enc_key in ['encz_last', 'encz_pre_last']:
    print(f'{model_enc_key}; kf={kf}; mid={mid}')
    model_name = 'mdl=SerialTimeModAttn~input_dims=1~dummy_seft=1~m=24~kernel_size=1~heads=8~time_noise_window=6*24**-1~enc_emb=128-128~dec_emb=g1-g128.r1-r128~b=203~pb=.~hr=0~bypass_synth=0~bypass_prob=0.0~ds_prob=0.1'
    filedir = f'../save/{model_name}/pre-training/dim_reductions/{cfilename}/{kf}@test/id={mid}~model_enc_key={model_enc_key}.d'
    d = files.load_pickle(filedir)
    thdays = d['thdays']

    plot_animator = PlotAnimator(f'../temp/exp=umap~kf={kf}~mid={mid}~model_enc_key={model_enc_key}.gif',
        save_frames=True,
        )
    axis_lims = AxisLims({'x':(None, None), 'y':(None, None)},
        axis_extended_percent=0.01,                
        )
    for thday in thdays:
        axis_lims.append('x', d['days_dim_reductions'][thday][:,0])
        axis_lims.append('y', d['days_dim_reductions'][thday][:,1])

    figsize = (8,8)
    dpi = 200
    for thday in thdays[::1]:
        x = d['days_dim_reductions'][thday]
        y = d['days_y_true'][thday]
        class_names = d['class_names']
        _, counts = np.unique(y, return_counts=True)
        fig, ax = plt.subplots(1, 1, figsize=figsize, dpi=dpi)
        scatter_kwargs = {}
        for kc,c in enumerate(class_names):
            scatter_kwargs[c] = {
                's':32*CLASSES_STYLES[c]['markerprop'],
                'color':get_scaled_color(CLASSES_STYLES[c]['c'], *2.5) if c=='SNIa' else CLASSES_STYLES[c]['c'],
                'marker':CLASSES_STYLES[c]['marker'],
                'linewidth':0,
                'label':c.replace('*', ''),
                }
        scatter(ax, x, y, class_names, scatter_kwargs, label_format='{label} ({count})')
        title = ''
        title += f'{utils.get_fmodel_name(model_name)}'+'\n'
        title += f'th-day={thday:.0f} [days]'+'\n'
        ax.set_title(title[:-1])
        ax.legend(loc='upper right')
        axis_lims.set_ax_axis_lims(ax)
        ax.grid(alpha=0.0)
        plot_animator.append(fig)
        fig.tight_layout()
        plt.show()
        #assert 0
    plot_animator.save()